# 데이터 로드

In [ ]:
#data load
import numpy as np 
import pandas as pd
items = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/items.csv")
submission_sample = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv")
cats = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv")
train = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
shops = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/shops.csv")
test = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")

# EDA / 전처리

In [ ]:
#카테고리 코드정리
#1. 문자열 분리
cats['split'] = cats['item_category_name'].str.split('-')
#2. [0]인 값만 사용, [1]의 unique()가 너무 많고, 최대 중복이 4여서 버림
cats['type'] = cats['split'].map(lambda x: x[0].strip())

#3. 카테고리 병합(PC와 모바일 게임도 게임으로 재분류)
cats.loc[cats['type']=='Игры PC','type'] = 'Игры'
cats.loc[cats['type']=='Игры MAC','type'] = 'Игры'
cats.loc[cats['type']=='Игры Android''type'] = 'Игры'

#4. 중복이 4보다 적은 값은 orters로 분류 -> label or onehot을 위해 최소화
cat_types =[]
for i in cats['type'].unique():
    if len(cats[cats['type'] == i]) >= 4:
        cat_types.append(i)
cats['type'] = cats['type'].apply(lambda x: x if (x in cat_types) else "orters")
#5. 라벨인코더 
from sklearn.preprocessing import LabelEncoder
hot_model = LabelEncoder()
cats['type_1'] = hot_model.fit_transform(cats['type'])

#6. 병합할 데이터 셋 
#병합시 Key(on) = item_category_id
cats_final = pd.DataFrame({'item_category_id':cats['item_category_id'],'type':cats['type_1']})


In [ ]:
#item name은 cats의 type과 성질이 유사하여 삭제
items = pd.merge(items,cats_final, on='item_category_id', how="left")
items = items.drop('item_name', axis=1)

In [ ]:
# train/test의 shop_id열에서 shop_name이 중복인 값의 shop_id 통일
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11
# shops의 shop_id열에서도 shop_name이 중복인 값의 shop_id 통일
shops.loc[shops.shop_id == 0, 'shop_id'] = 57
shops.loc[shops.shop_id == 0, 'shop_id'] = 57
shops.loc[shops.shop_id == 1, 'shop_id'] = 58
shops.loc[shops.shop_id == 1, 'shop_id'] = 58
shops.loc[shops.shop_id == 10, 'shop_id'] = 11
shops.loc[shops.shop_id == 10, 'shop_id'] = 11
# shop_name에서 분류가 안되는 값들 분류가 잘 되도록 수정하고, (city나 category명 없거나 띄어쓰기 이상하게 되있거나 한 값들이 꽤 있어서)
shops.iloc[0,0] = 'Якутск Орджоникидзе, 56 фран	'
shops.iloc[1,0] = 'Якутск ТЦ "Центральный" фран'
shops.iloc[6,0] = 'Воронеж  (Плехановская, 13)'
shops.iloc[9,0] = ' Выездная Торговля '
shops.iloc[10,0] = 'Жуковский  ул.Чкалова39м'
shops.iloc[11,0] = 'Жуковский  ул.Чкалова39м²'
shops.iloc[12,0] = ' Интернет-магазин ЧС'
shops.iloc[20,0] = 'Москва  "Распродажа"'
shops.iloc[46,0] = 'СергиевПосад ТЦ "7Я"'
shops.iloc[55,0] = ' Цифровойсклад1С-Онлайн '
shops.iloc[57,0] = 'Якутск  Орджоникидзе'
shops['category'] = shops.shop_name.str.split(' ').map(lambda x: x[1])
shops["city"] = shops.shop_name.str.split(" ").map( lambda x: x[0] )
# shop_id에서 id가 중복되는 행 삭제 (미정)
shops.drop_duplicates(['shop_id'],inplace = True)
# categoty 항목이 5개 미만이거나 공백인 값들은 other로 변경
category = []
for cat in shops.category.unique():
    if len(shops[shops.category == cat]) >= 5 and cat != '':
        category.append(cat)
shops.category = shops.category.apply( lambda x: x if (x in category) else "other" )
# ["shop_id", "shop_category", "shop_city"] 행만 남게 한 후 라벨 인코딩
from sklearn.preprocessing import LabelEncoder
shops['shop_category'] = LabelEncoder().fit_transform(shops.category)
shops['shop_city'] = LabelEncoder().fit_transform(shops.city)
shops = shops[["shop_id", "shop_category", "shop_city"]]

In [ ]:
#outlier 정리
train = train[(train['item_price'] < 100000) & (train['item_price'] > 0)& (train['item_cnt_day'] <1000)]

In [ ]:
#product = 최대순열 만드는 함수
from itertools import product
matrix = []
cols = ['date_block_num','shop_id','item_id']
matrix = []
for i in range(34):
    sales = train[train.date_block_num == i]
    matrix.append( np.array(list( product( [i], sales.shop_id.unique(), sales.item_id.unique() ) ), dtype = np.int16) )

matrix = pd.DataFrame( np.vstack(matrix), columns = ["date_block_num", "shop_id", "item_id"] )
matrix["date_block_num"] = matrix["date_block_num"].astype(np.int8)
matrix["shop_id"] = matrix["shop_id"].astype(np.int8)
matrix["item_id"] = matrix["item_id"].astype(np.int16)
matrix.sort_values( ["date_block_num", "shop_id", "item_id"], inplace = True )

In [ ]:
group = train.groupby( ["date_block_num", "shop_id", "item_id"] ).agg( {"item_cnt_day": ["sum"]} )
group.columns = ["item_cnt_month"]
group.reset_index( inplace = True)
matrix = pd.merge( matrix, group, on = ["date_block_num", "shop_id", "item_id"], how = "left" )
matrix["item_cnt_month"] = matrix["item_cnt_month"].fillna(0).astype(np.float16)

In [ ]:
test["date_block_num"] = 34
test["date_block_num"] = test["date_block_num"].astype(np.int8)
test["shop_id"] = test.shop_id.astype(np.int8)
test["item_id"] = test.item_id.astype(np.int16)
matrix = pd.concat([matrix, test.drop(["ID"],axis = 1)], ignore_index=True, sort=False, keys=["date_block_num", "shop_id", "item_id"])
matrix.fillna( 0, inplace = True )

In [ ]:
matrix = pd.merge( matrix, shops, on = ["shop_id"], how = "left" )
matrix = pd.merge(matrix, items, on = ["item_id"], how = "left")
matrix.info()

In [ ]:
matrix["shop_category"] = matrix["shop_category"].astype(np.int8)
matrix["shop_city"] = matrix["shop_city"].astype(np.int8)
matrix["type"] = matrix["type"].astype(np.int8)

In [ ]:
group = train.groupby(['date_block_num',"item_id"])['item_price'].mean()
group.columns = ["item_price"]
# group.reset_index( inplace = True)
matrix = pd.merge(matrix, group, on = ['date_block_num',"item_id"], how = "left" )
matrix["item_price"] = matrix["item_price"].fillna(0).astype(np.float16)

In [ ]:
matrix["revenue"] = matrix["item_cnt_month"] * matrix["item_price"]

In [ ]:
item_id_list = matrix['item_id'].unique()
lis_price = {}
for i in item_id_list:
    pwd = matrix[matrix['item_id']==i]['item_price']
    lis_price[i]=len(pwd.value_counts())
item_id_list = matrix['item_id'].unique()
item_id_list.sort()
item_id_list
sample = []
for i in item_id_list:
    sample.append(lis_price[i])
price_merge_sample = pd.DataFrame({'item_id':np.array(item_id_list),'price_unique':np.array(sample)})

matrix = pd.merge(matrix, price_merge_sample, on = ["item_id"], how = "left" )
matrix["price_unique"] = matrix["price_unique"].fillna(0).astype(np.int16)

In [ ]:
matrix['Dec_dummy'] = matrix['date_block_num'].apply(lambda x: 1 if x%12==0 else 0)

matrix["item_category_id"] = matrix["item_category_id"].fillna(0).astype(np.int16)
matrix["Dec_dummy"] = matrix["Dec_dummy"].fillna(0).astype(np.int8)

In [ ]:
#lag 함수 만들기
def lag_data( df,lags, cols ):
    for col in cols:
        print(col)
        tmp = df[["date_block_num", "shop_id","item_id",col ]]
        for i in lags:
            shifted = tmp.copy()
            shifted.columns = ["date_block_num", "shop_id", "item_id", col + "_lag_"+str(i)]
            shifted.date_block_num = shifted.date_block_num + i
            df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

In [ ]:
matrix = lag_data(matrix, [1,2,3], ['item_cnt_month'])

In [ ]:
matrix = lag_data(matrix, [1,2,3], ["revenue"])

In [ ]:
#월평균 아이템 가격의 lag다.
matrix = lag_data(matrix, [1,2,3], ["item_price"])

In [ ]:
matrix = lag_data(matrix, [1], ["Dec_dummy"])

In [ ]:
data = matrix.copy()

In [ ]:
X_train = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)
Y_train = Y_train.clip(0, 20)
Y_valid = Y_valid.clip(0, 20)

In [ ]:
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
lgb_model = lgb.LGBMRegressor()

params = {'task':['predict'],
#           'max_depth':[10,25,50,100]
          'boosting_type':['gbdt', 'dart', 'rf'],
          'learning_rate':[0.1, 0.5],
         'n_estimators':[1000,2000,3000]}
#https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html

gv_model = GridSearchCV(estimator=lgb_model, param_grid=params)

gv_model.fit(X_train, Y_train)

In [ ]:
gv_model.best_params_

In [ ]:
model = gv_model.best_estimator_

In [ ]:
model.fit(
    X_train, 
    Y_train, 
    eval_metric="rmse",
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
    verbose=True, 
    early_stopping_rounds = 20)